# Реализация классов

In [ ]:
from datetime import datetime


class Account:
    """Базовый банковский счёт: пополнение, снятие и история операций."""
    def __init__(self, account_holder, balance=0):
        if balance < 0:
            raise ValueError("Баланс счёта не может быть отрицательным")

        self.holder = account_holder
        self._balance = balance
        self.operations_history = []

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Сумма пополнения должна быть положительной")

        self._balance += amount
        self._add_operation('deposit', amount, 'success')

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной")

        # В случае недостатка средств фиксируем в истории неудачный запрос.
        if self._balance < amount:
            self._add_operation('withdraw', amount, 'fail')
        else:
            self._balance -= amount
            self._add_operation('withdraw', amount, 'success')

    def get_balance(self):
        return self._balance

    def get_history(self):
        formatted = []

        for operation in self.operations_history:
            formatted.append({
                **operation,
                "datetime": operation["datetime"].strftime("%d.%m.%Y %H:%M:%S")
            })

        return formatted

    def _add_operation(self, operation_type, amount, status):
        # Добавляем новую операцию в историю, используя словарь.
        operation = {
            "type": operation_type,
            "amount": amount,
            "datetime": datetime.now(),
            "balance_after": self._balance,
            "status": status
        }

        self.operations_history.append(operation)


class CreditAccount(Account):
    """Счёт с кредитным лимитом: баланс может уходить в минус до -credit_limit."""
    def __init__(self, credit_limit, account_holder, balance=0):
        if credit_limit <= 0:
            raise ValueError("Кредитный лимит должен быть положительным")

        super().__init__(account_holder, balance)
        self._credit_limit = credit_limit

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной")

        new_balance = self._balance - amount

        # В случае недостатка средств с учетом кредитного лимита
        # фиксируем в истории неудачный запрос.
        if new_balance < -self._credit_limit:
            self._add_operation('withdraw', amount, 'fail')
        else:
            # Если баланс после обновления стал отрицательным, значит,
            # использовались кредитные средства.
            self._balance = new_balance
            self._add_operation(
                'withdraw',
                amount,
                'success',
                used_credit=self._balance < 0,
            )

    def get_available_credit(self):
        # Возвращаем, сколько кредитных средств еще доступно.
        return self._balance + self._credit_limit

    def _add_operation(
        self,
        operation_type,
        amount,
        status,
        used_credit=False,
    ):
        # Добавляем операцию с доп. полем, отвечающем за информацию о том,
        # были ли использованы кредитные средства.
        operation = {
            "type": operation_type,
            "amount": amount,
            "datetime": datetime.now(),
            "balance_after": self._balance,
            "status": status,
            "used_credit": used_credit,
        }

        self.operations_history.append(operation)


# Использование классов

In [ ]:
account = Account("A")

print(account.get_balance())
account.deposit(100)
print(account.get_balance())
account.withdraw(200)
print(account.get_balance())
account.withdraw(50)
print(account.get_balance())

print(*account.get_history(), sep='\n')

0
100
100
50
{'type': 'deposit', 'amount': 100, 'datetime': '27.12.2025 14:23:38', 'balance_after': 100, 'status': 'success'}
{'type': 'withdraw', 'amount': 200, 'datetime': '27.12.2025 14:23:38', 'balance_after': 100, 'status': 'fail'}
{'type': 'withdraw', 'amount': 50, 'datetime': '27.12.2025 14:23:38', 'balance_after': 50, 'status': 'success'}


In [ ]:
cr_account = CreditAccount(500, "B", 200)

print(cr_account.get_available_credit())
print(cr_account.get_balance())
cr_account.deposit(100)
print(cr_account.get_balance())
cr_account.withdraw(2000)
print(cr_account.get_balance())
cr_account.withdraw(750)
print(cr_account.get_balance())

print(*cr_account.get_history(), sep='\n')

700
200
300
300
-450
{'type': 'deposit', 'amount': 100, 'datetime': '27.12.2025 14:23:41', 'balance_after': 300, 'status': 'success', 'used_credit': False}
{'type': 'withdraw', 'amount': 2000, 'datetime': '27.12.2025 14:23:41', 'balance_after': 300, 'status': 'fail', 'used_credit': False}
{'type': 'withdraw', 'amount': 750, 'datetime': '27.12.2025 14:23:41', 'balance_after': -450, 'status': 'success', 'used_credit': True}


In [ ]:
# Account
account = Account("A", balance=100)

account.deposit(50)
assert account.get_balance() == 150
last = account.get_history()[-1]
assert last["type"] == "deposit"
assert last["amount"] == 50
assert last["status"] == "success"
assert last["balance_after"] == 150

account.withdraw(70)
assert account.get_balance() == 80
last = account.get_history()[-1]
assert last["type"] == "withdraw"
assert last["amount"] == 70
assert last["status"] == "success"
assert last["balance_after"] == 80

account.withdraw(1000)
assert account.get_balance() == 80
last = account.get_history()[-1]
assert last["type"] == "withdraw"
assert last["amount"] == 1000
assert last["status"] == "fail"
assert last["balance_after"] == 80

# CreditAccount
cr_account = CreditAccount(credit_limit=200, account_holder="B",
                               balance=100)

cr_account.withdraw(250)
assert cr_account.get_balance() == -150
last = cr_account.get_history()[-1]
assert last["type"] == "withdraw"
assert last["amount"] == 250
assert last["status"] == "success"
assert last["balance_after"] == -150
assert last["used_credit"] is True

cr_account.withdraw(100)
assert cr_account.get_balance() == -150
last = cr_account.get_history()[-1]
assert last["status"] == "fail"
assert last["balance_after"] == -150
assert last["used_credit"] is False

assert cr_account.get_available_credit() == cr_account.get_balance() + 200